In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('Bank_Personal_Loan_Modelling.csv')

In [4]:
x = data.iloc[:, [1,2,3,5,6,7,8,10,11,12,13]].values
y = data.iloc[:, [9]].values
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [5]:
ins = 11
hns = 6
ous = 1

In [6]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [7]:
def fitness_function(x_train, y_train, weights):
    hn_output = sigmoid(np.dot(x_train,weights[0]))
    out = sigmoid(np.dot(hn_output, weights[1]))
    out_error = y_train - out
    mse = 1 + np.mean(np.square(out_error))
    return 1 / mse

In [8]:
def genetic(x_train, y_train, population, num):
    population1 = population
    pop_size = len(population1)
    for i in range(num):
        fitness = []
        for weight in population1:
            fitness.append(fitness_function(x_train, y_train, weight))
        fitness = np.array(fitness)
        
        parents = []
        for i in range(2):
            parent_idx = np.random.choice(pop_size, p=fitness/(fitness.sum()))
            parents.append(population1[parent_idx])
        children = []
        for i in range(pop_size):
            weights1 = parents[0][0] * np.random.rand(ins, hns) + parents[1][0] * (1 - np.random.rand(ins, hns))
            weights2 = parents[0][1] * np.random.rand(hns, ous) + parents[1][1] * (1 - np.random.rand(hns, ous))
            children.append([weights1, weights2])

        for j in range(pop_size):
            if np.random.rand() < 0.1:
                children[j][0] += np.random.randn(ins, hns) * 0.1
            if np.random.rand() < 0.1:
                children[j][1] += np.random.randn(hns,ous) * 0.1
                
        population1 = children
    
    opti_weight = []
    opti_fitness = 0
    for _ in population1:
        fitness = fitness_function(x_train, y_train, _)
        if fitness > opti_fitness:
            opti_weight = _
            opti_fitness = fitness
    return opti_weight

In [148]:
num_of_generations = 80
population = []
for i in range(10):
    weights1 = np.random.randn(ins, hns)
    weights2 = np.random.randn(hns, ous)
    population.append((weights1, weights2))

In [149]:
oweight = genetic(x_test, y_test, population, num_of_generations)

In [150]:
out1 = sigmoid(np.dot(x_test, oweight[0])) #neural network
out2 = sigmoid(np.dot(out1, oweight[1]))

In [151]:
y_pred = out2.round()

In [153]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.9128

In [14]:
numi = 100
nump = 5
numw = (ins*hns) + (hns*ous)

In [15]:
def PSO(particle, velocity,x,y, w, nump, numi, numw, c1, c2):
    opti_fitness = 0
    opti_weight = []
    for i in range(numi):
        for j in range(nump):
            p = particle[j]
            v = velocity[j]
            fitness = fitness_function(x, y, p)
            if fitness > opti_fitness:
                opti_fitness = fitness
                opti_weight = np.copy(p)
            for k in range(numw):
                r1 = np.random.rand()
                r2 = np.random.rand()
                v[k] = (w*v[k]) + c1*r1*(opti_weight[k] - p[k]) + c2*r2*(p[k] - opti_weight[k])
                p[k] +=v[k]
            velocity[j] = v
            particle[j] = p
    return [np.array(opti_weight[:ins * hns]).reshape((ins, hns)),
            np.array(opti_weight[ins * hns:]).reshape((hns, ous))]

In [16]:
particle = []
velocity = []
for i in range(nump):
    p = []
    v = []
    for j in range(numw):
        p.append(np.random.rand())
        v.append(0)
    particle.append(p)
    velocity.append(v)

In [91]:
oweight = PSO(particle, velocity,x_test, y_test, 0.5, nump, numi, numw, 0.5,0.5)

In [123]:
out1 = sigmoid(np.dot(x_test, oweight[0])) #neural network
out2 = sigmoid(np.dot(out1, oweight[1]))
y_pred = out2.round()
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

C:\Users\vscse\AppData\Local\Temp/ipykernel_11156/91620759.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1+np.exp(-x))


0.9072

In [101]:
def ACO(num_ants, numi, numw, x, y, alpha, beta, evaporation_rate, pheromone_deposit):
    pheromone_trail = np.ones((numw, num_ants))
    gopti_pos = []
    gopti_fitness = 0
    for _ in range(numi):
        ant_position = np.zeros((num_ants, numw))
        ant_fitness = np.zeros(num_ants)
        for i in range(num_ants):
            current_idx = np.random.randint(numw)
            ant_position[i][current_idx] = np.random.uniform(-1, 1)
            for j in range(current_idx + 1, numw):
                phweight = pheromone_trail[j] ** alpha
                pweight = (phweight) ** beta
                sump=0
                for i in range(len(pweight)):
                    sump += pweight[i]
                if sump!=0:
                    pweight = pweight/sump
                selected_idx = np.random.choice(num_ants, p = pweight)
                ant_position[i][j] = ant_position[selected_idx][j]
            ant_fitness[i] = fitness_function(x, y,[np.array(ant_position[i][:ins * hns]).reshape((ins, hns)), np.array(ant_position[i][ins * hns:]).reshape((hns, ous))])
            if ant_fitness[i] > gopti_fitness: #Cost Function
                gopti_pos = ant_position[i].copy()
                gopti_fitness = ant_fitness[i]
        pheromone_trail *= evaporation_rate #Decay Rate
        for ant in range(num_ants):
            for weight_index in range(numw):
                pheromone_trail[j][i] += pheromone_deposit / ant_fitness[i] if ant_fitness[i] > 0 else 0 #With respect to importance of ant
    return [np.array(gopti_pos[:ins * hns]).reshape((ins, hns)),
            np.array(gopti_pos[ins * hns:]).reshape((hns, ous))]

In [160]:
num_ants = 20
oweight = ACO(num_ants, 100, numw ,x_test, y_test, 1,1, 0.5, 100)

In [107]:
out1 = sigmoid(np.dot(x_test, oweight[0])) #neural network
out2 = sigmoid(np.dot(out1, oweight[1]))
y_pred = out2.round()
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.9136

In [183]:
def cultural_algorithm(numi, numw, x, y, numl, numb, numn, mutation_rate, mutation_stddev):
    belief = np.random.uniform(-1, 1, (numb, numw))
    norm = np.random.uniform(0, 1, (numn, numw))
    gopti_pos = []
    opti_fitness = 0
    for iteration in range(numi):
        learner_position = np.zeros((numl,numw))
        learner_fitness = np.zeros(numl)
        for i in range(numl):
            selected_idx = np.random.randint(numb)
            learner_position[i] = belief[selected_idx].copy()
            for j in range(len(learner_position[i])):
                if np.random.rand() < mutation_rate:
                    learner_position[i][j] += np.random.normal(0, mutation_stddev)
            learner_fitness[i] = fitness_function(x,y,[np.array(learner_position[i][:ins * hns]).reshape((ins, hns)),
                                                           np.array(learner_position[i][ins * hns:]).reshape((hns, ous))])
            if learner_fitness[i] > opti_fitness:
                gopti_pos = learner_position[i].copy()
                opti_fitness = learner_fitness[i]
        for belief_index in range(numb):
            belief_learner = [learner_position[j] for j in range(numl) if norm[j][belief_index] > np.mean(norm[:, belief_index])]
            if belief_learner:
                belief[belief_index] = np.mean(belief_learner, axis=0)
            for norm_index in range(numn):
                if np.random.rand() < 0.5:
                    norm[norm_index][belief_index] += np.random.normal(0, 0.1)
                norm[norm_index][belief_index] = np.clip(norm[norm_index][belief_index], 0, 1)
                
    return [np.array(gopti_pos[:ins * hns]).reshape((ins, hns)),
            np.array(gopti_pos[ins * hns:]).reshape((hns, ous))]

In [220]:
numi = 100
numl = 15
numb = 18
numn = 20
oweight = cultural_algorithm(numi, numw, x_test, y_test, numl, numb, numn,0.01, 0.01)

In [221]:
out1 = sigmoid(np.dot(x_test, oweight[0])) #neural network
out2 = sigmoid(np.dot(out1, oweight[1]))
y_pred = out2.round()
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.9032